<h1><b> Coursera Capstone Project </h1>

This notebook contains the code for the Coursera Capstone Project on Data Science by IBM and Coursera

<h4><b> Importing required libraries </h4>

In [1]:
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium --yes
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import requests
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from folium import plugins
from folium import features

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



<h4><b> Setting maxcolwidth to 800 for good viewability</h4>

In [2]:
pd.set_option('max_colwidth', 800)

<h2><b> Average Yearly Price per m2 per Canton - Switzerland, 2019 </h2>

<h3><b> Data Wrangling </h3>

<h4><b> Reading the downloaded Excel database </h4>

In [3]:
CH = pd.read_excel('2 Canton Data.xlsx')
CH.head()

,Canton,Canton Number,Population,Average price / m2 (Apartment),Average price / m2 (House),Average price / m2 (Rent yearly)
0,Canton of Aargau,19,"670,988","5,960","6,133",223
1,Canton of Appenzell Ausserrhoden,15,"55,178","5,380","5,171",196
2,Canton of Appenzell Innerrhoden,16,"16,105","7,223","6,158",210
3,Canton of Basel-Landschaft,13,"287,023","7,499","6,485",249
4,Canton of Basel-Stadt,12,"193,908","9,538","8,444",286


<h4><b> Changing the column names & modifying the 'Canton' values </h4>

In [4]:
CH.rename(columns={'Average price / m2 (Apartment)':'Apartment Price', 'Average price / m2 (House)':'House Price','Average price / m2 (Rent yearly)':'Rent Yearly'},inplace=True)
CH.drop(['Population'], axis=1, inplace=True)
CH.head()

,Canton,Canton Number,Apartment Price,House Price,Rent Yearly
0,Canton of Aargau,19,"5,960","6,133",223
1,Canton of Appenzell Ausserrhoden,15,"5,380","5,171",196
2,Canton of Appenzell Innerrhoden,16,"7,223","6,158",210
3,Canton of Basel-Landschaft,13,"7,499","6,485",249
4,Canton of Basel-Stadt,12,"9,538","8,444",286


In [5]:
CH[['None','Kanton']] = CH.Canton.str.split("Canton of",expand=True,)
Kanton = CH.drop(['Canton','None'], axis=1)
Kanton.head()

,Canton Number,Apartment Price,House Price,Rent Yearly,Kanton
0,19,"5,960","6,133",223,Aargau
1,15,"5,380","5,171",196,Appenzell Ausserrhoden
2,16,"7,223","6,158",210,Appenzell Innerrhoden
3,13,"7,499","6,485",249,Basel-Landschaft
4,12,"9,538","8,444",286,Basel-Stadt


<h4><b> Creating the map of Switzerland with Rent/m2 distribution within Kantons </h4>

In [6]:
SwissMap = folium.Map(width=900, height=640, location=[46.8, 8.33], zoom_start=8, tiles='Mapbox Bright')

chor = folium.Choropleth(
    geo_data='Swiss Map.geojson',
    name='choropleth',
    data=Kanton,
    columns=['Canton Number', 'Rent Yearly'],
    key_on='feature.properties.KANTONSNUM',
    fill_color='YlOrRd', 
    fill_opacity=1.0, 
    line_opacity=0.7,
    legend_name='Rent per m2 per Kanton (yearly)'
     ).add_to(SwissMap)

In [7]:
SwissMap

<h4><b> Adding markers for 5 most expensive Kantons + Kanton name's display when hovered over it </h4>

In [8]:
Kanton5 = Kanton.sort_values(by='Rent Yearly',ascending=False)
Kanton5 = Kanton5.head(5)
Kanton5

,Canton Number,Apartment Price,House Price,Rent Yearly,Kanton
9,18,"7,273","4,298",485,Grisons
7,25,"11,928","12,391",390,Geneva
24,9,"11,037","10,601",313,Zug
25,1,"8,318","8,049",296,Zurich
23,22,"7,822","7,814",288,Vaud


In [9]:
data = pd.DataFrame({
'lat':[46.8508, 46.2044, 47.1662, 47.3769, 46.5197],
'lon':[9.532,6.1432,8.5155,8.5417,6.62323],
'name':['Grisons CHF485', 'Geneva CHF390', 'Zug CHF313', 'Zurich CHF296', 'Vaud CHF288']
})

for i in range(0,len(data)):
    folium.Marker([data.iloc[i]['lat'], data.iloc[i]['lon']], popup=data.iloc[i]['name']).add_to(SwissMap)
    
chor.geojson.add_child(
    folium.features.GeoJsonTooltip(['NAME'],labels=False)
)

In [10]:
SwissMap

<h2><b> Average Yearly Price per m2 per Neighborhood - Canton of Zug, Switzerland, 2019 </h2>

<h4><b> Changing the column names </h4>

In [11]:
Zug = pd.read_excel('3 Zug Data.xlsx')
Zug.head()

,Municipality,Zip,Gemeinden,Average price / m2 (Apartment),Average price / m2 (House),Average price / m2 (Rent yearly),Latitude,Longitude
0,Risch,6343,Risch,NaN,"10,556",660,47.13474,8.46636
1,Zug,6300,Zugerberg,"16,201","12,632",535,47.14359,8.53673
2,Menzingen,6313,Edlibach,"9,067","10,264",388,47.18105,8.56541
3,Walchwil,6318,Walchwil,"11,583","11,193",370,47.09960,8.51657
4,Zug,6300,Zug,"9,197","2,809",368,47.17158,8.51631


In [12]:
Zug.rename(columns={'Average price / m2 (Apartment)':'Apartment Price', 'Average price / m2 (House)':'House Price','Average price / m2 (Rent yearly)':'Rent Yearly'},inplace=True)
Zug.head()

,Municipality,Zip,Gemeinden,Apartment Price,House Price,Rent Yearly,Latitude,Longitude
0,Risch,6343,Risch,NaN,"10,556",660,47.13474,8.46636
1,Zug,6300,Zugerberg,"16,201","12,632",535,47.14359,8.53673
2,Menzingen,6313,Edlibach,"9,067","10,264",388,47.18105,8.56541
3,Walchwil,6318,Walchwil,"11,583","11,193",370,47.09960,8.51657
4,Zug,6300,Zug,"9,197","2,809",368,47.17158,8.51631


<h4><b> Map plotted with Zug Gemeinden using Folium </h4>

In [16]:
address = 'Zug, Switzerland'

geolocator = Nominatim(user_agent="ch_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Zug are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Zug are 47.1486137, 8.5539378.


In [17]:
Map_Zug = folium.Map(width=900, height=640, location=[latitude, longitude], zoom_start=11.5, tiles='cartodbpositron')

for lat, lng, mun, gem in zip(Zug['Latitude'], Zug['Longitude'], Zug['Municipality'], Zug['Gemeinden']):
    label = '{}, {}'.format(gem, mun)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Map_Zug)  
    
Map_Zug

<h2><b> 5 Gemeinden with the highest Gross Rent / m2 in Zug Kanton </h2>

In [14]:
Zug.sort_values('Rent Yearly',ascending=False)
Zug5 = Zug.head(5)
Zug5

,Municipality,Zip,Gemeinden,Apartment Price,House Price,Rent Yearly,Latitude,Longitude
0,Risch,6343,Risch,NaN,"10,556",660,47.13474,8.46636
1,Zug,6300,Zugerberg,"16,201","12,632",535,47.14359,8.53673
2,Menzingen,6313,Edlibach,"9,067","10,264",388,47.18105,8.56541
3,Walchwil,6318,Walchwil,"11,583","11,193",370,47.09960,8.51657
4,Zug,6300,Zug,"9,197","2,809",368,47.17158,8.51631


In [18]:
Map_Zug5 = folium.Map(width=900, height=640, location=[latitude, longitude], zoom_start=12, tiles='cartodbpositron')

for lat, lng, gem, rent in zip(Zug5['Latitude'], Zug5['Longitude'], Zug5['Gemeinden'],Zug5['Rent Yearly']):
    label = '{}, {}'.format(gem, rent)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Map_Zug5)  
    
Map_Zug5

<h2><b> Using Foursquare to explore the venues around selected Gemeindens in Zug </h2>

In [19]:
CLIENT_ID = #
CLIENT_SECRET = #
VERSION = #
LIMIT = 30

<h4><b> Testing the functionality on Example 1 </h4>

In [20]:
gemeinden_latitude = Zug5.loc[0, 'Latitude']
gemeinden_longitude = Zug5.loc[0, 'Longitude']

gemeinden_name = Zug5.loc[0, 'Gemeinden']

print('Latitude and longitude values of {} are {}, {}.'.format(gemeinden_name, 
                                                               gemeinden_latitude, 
                                                               gemeinden_longitude))

Latitude and longitude values of Risch are 47.13474, 8.46636.


In [21]:
LIMIT = 100
radius = 2000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    gemeinden_latitude, 
    gemeinden_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=0Q3XPRNAPEOOJPAT0XOOKGDPRKZPHOIMZCDC1CZ5Z5OPGX21&client_secret=4DS5QIY20CFMPYYKNVFKLVAUL5WUFR3JAX5RUWYIBH5AVKAX&v=20180604&ll=47.13474,8.46636&radius=2000&limit=100'

<h4><b> Saving the results into the JSON file </h4>

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed814bd1e152c001b4b492d'},
 'response': {'headerLocation': 'Risch',
  'headerFullLocation': 'Risch',
  'headerLocationGranularity': 'city',
  'totalResults': 7,
  'suggestedBounds': {'ne': {'lat': 47.15274001800002,
    'lng': 8.492770448859552},
   'sw': {'lat': 47.116739981999984, 'lng': 8.439949551140447}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bddb32ce75c0f472f3ec503',
       'name': 'Buonas Beach',
       'location': {'address': 'Seestrasse',
        'lat': 47.14247476601744,
        'lng': 8.458099365234375,
        'labeledLatLngs': [{'label': 'display',
          'lat': 47.14247476601744,
          'lng': 8.458099365234375}],
        'distance': 1064,
        'cc': 'CH',
        'city': 'Buonas',
        'state': '

<h4><b> Extracting the category of the venues </h4>

In [25]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

<h4><b> Dataframe creation + modification </h4>

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Buonas Beach,Beach,47.142475,8.458099
1,Bowling Center White Line,Bowling Alley,47.125175,8.449997
2,Landgasthof Breitfeld,Restaurant,47.132852,8.444535
3,Hotel Waldheim Risch AG,Hotel,47.133532,8.465935
4,Restaurant Wildenmann,Seafood Restaurant,47.142241,8.456614


<h4><b> Creating a FourSquare API to repeat the same process for all top 5 Gemeindens in Zug</h4>

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Gemeinden', 
                  'Gemeinden Latitude', 
                  'Gemeinden Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h4><b> Running the API for all Gemeindens within the Zug5 dataframe</h4>

In [28]:
Zug5_venues = getNearbyVenues(names=Zug5['Gemeinden'],
                                   latitudes=Zug5['Latitude'],
                                   longitudes=Zug5['Longitude'])

Risch
Zugerberg
Edlibach
Walchwil
Zug


<h4><b> Getting the total number of venues</h4>

In [29]:
print(Zug5_venues.shape)
Zug5_venues.head()

(124, 7)


,Gemeinden,Gemeinden Latitude,Gemeinden Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Risch,47.13474,8.46636,Buonas Beach,47.142475,8.458099,Beach
1,Risch,47.13474,8.46636,Bowling Center White Line,47.125175,8.449997,Bowling Alley
2,Risch,47.13474,8.46636,Landgasthof Breitfeld,47.132852,8.444535,Restaurant
3,Risch,47.13474,8.46636,Hotel Waldheim Risch AG,47.133532,8.465935,Hotel
4,Risch,47.13474,8.46636,Restaurant Wildenmann,47.142241,8.456614,Seafood Restaurant


<h4><b> Creating a dataframe for performing clustering operations </h4>

In [30]:
Zug5_venues.groupby('Gemeinden').count()

,Gemeinden Latitude,Gemeinden Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Gemeinden,,,,,,
Edlibach,9,9,9,9,9,9
Risch,7,7,7,7,7,7
Walchwil,5,5,5,5,5,5
Zug,95,95,95,95,95,95
Zugerberg,8,8,8,8,8,8


In [31]:
print('There are {} uniques categories.'.format(len(Zug5_venues['Venue Category'].unique())))

There are 62 uniques categories.


<h4><b> Performing one-hot encoding to analyze each Gemeinden</h4>

In [32]:
Zug5_onehot = pd.get_dummies(Zug5_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Zug5_onehot['Gemeinden'] = Zug5_venues['Gemeinden'] 

# move neighborhood column to the first column
fixed_columns = [Zug5_onehot.columns[-1]] + list(Zug5_onehot.columns[:-1])
Zug5_onehot = Zug5_onehot[fixed_columns]

Zug5_onehot.head()

,Gemeinden,Asian Restaurant,BBQ Joint,Bakery,Bar,Beach,Bistro,Bookstore,Bowling Alley,Breakfast Spot,...,Supermarket,Sushi Restaurant,Swiss Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Waterfront,Wine Shop
0,Risch,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Risch,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Risch,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Risch,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Risch,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h4><b> Aggregating venues by Gemeinden</h4>

In [33]:
Zug5_grouped = Zug5_onehot.groupby('Gemeinden').mean().reset_index()
Zug5_grouped 

,Gemeinden,Asian Restaurant,BBQ Joint,Bakery,Bar,Beach,Bistro,Bookstore,Bowling Alley,Breakfast Spot,...,Supermarket,Sushi Restaurant,Swiss Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Waterfront,Wine Shop
0,Edlibach,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
1,Risch,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.142857,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
2,Walchwil,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.600000,0.000000,0.000000
3,Zug,0.010526,0.021053,0.021053,0.031579,0.031579,0.010526,0.010526,0.000000,0.010526,...,0.052632,0.031579,0.073684,0.010526,0.010526,0.010526,0.000,0.021053,0.010526,0.010526
4,Zugerberg,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000


<h4><b> Creating a function to return the most common venues </h4>

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<h4><b> Creating a new dataframe to get the top 5 venues </h4>

In [36]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Gemeinden']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Gemeinden_venues_sorted = pd.DataFrame(columns=columns)
Gemeinden_venues_sorted['Gemeinden'] = Zug5_grouped['Gemeinden']

for ind in np.arange(Zug5_grouped.shape[0]):
    Gemeinden_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Zug5_grouped.iloc[ind, :], num_top_venues)

Gemeinden_venues_sorted.head()

,Gemeinden,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Edlibach,Swiss Restaurant,Italian Restaurant,Bakery,Restaurant,Outdoors & Recreation
1,Risch,Lake,Bowling Alley,Hotel,Restaurant,Seafood Restaurant
2,Walchwil,Train Station,Grocery Store,Restaurant,Gas Station,College Gym
3,Zug,Swiss Restaurant,Hotel,Restaurant,Supermarket,Italian Restaurant
4,Zugerberg,Swiss Restaurant,Trail,Light Rail Station,Hotel,Restaurant


<h4><b> Using K-Means clustering </h4>

In [41]:
# set number of clusters
kclusters = 3

Zug5_grouped_clustering = Zug5_grouped.drop('Gemeinden', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Zug5_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 1, 0, 0], dtype=int32)

<h4><b> Creating a grand dataframe containing the necessary data </h4>

In [42]:
Gemeinden_venues_sorted.drop('Cluster Labels',axis=1, inplace=True)

In [43]:
# add clustering labels
Gemeinden_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Zug5_merged = Zug5

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Zug5_merged = Zug5_merged.join(Gemeinden_venues_sorted.set_index('Gemeinden'), on='Gemeinden')

Zug5_merged.head() # check the last columns!

,Municipality,Zip,Gemeinden,Apartment Price,House Price,Rent Yearly,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Risch,6343,Risch,NaN,"10,556",660,47.13474,8.46636,2,Lake,Bowling Alley,Hotel,Restaurant,Seafood Restaurant
1,Zug,6300,Zugerberg,"16,201","12,632",535,47.14359,8.53673,0,Swiss Restaurant,Trail,Light Rail Station,Hotel,Restaurant
2,Menzingen,6313,Edlibach,"9,067","10,264",388,47.18105,8.56541,0,Swiss Restaurant,Italian Restaurant,Bakery,Restaurant,Outdoors & Recreation
3,Walchwil,6318,Walchwil,"11,583","11,193",370,47.09960,8.51657,1,Train Station,Grocery Store,Restaurant,Gas Station,College Gym
4,Zug,6300,Zug,"9,197","2,809",368,47.17158,8.51631,0,Swiss Restaurant,Hotel,Restaurant,Supermarket,Italian Restaurant


<h4><b> Visualizing the clusters </h4>

In [53]:
# create map
map_clusters = folium.Map(width=900, height=640, location=[latitude, longitude], zoom_start=12,tiles='cartodbpositron')

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Zug5_merged['Latitude'], Zug5_merged['Longitude'], Zug5_merged['Gemeinden'], Zug5_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h4><b> Examining the clusters </h4>

In [48]:
Zug5_merged.loc[Zug5_merged['Cluster Labels'] == 0, Zug5_merged.columns[[2] + list(range(5, Zug5_merged.shape[1]))]]

,Gemeinden,Rent Yearly,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Zugerberg,535,47.14359,8.53673,0,Swiss Restaurant,Trail,Light Rail Station,Hotel,Restaurant
2,Edlibach,388,47.18105,8.56541,0,Swiss Restaurant,Italian Restaurant,Bakery,Restaurant,Outdoors & Recreation
4,Zug,368,47.17158,8.51631,0,Swiss Restaurant,Hotel,Restaurant,Supermarket,Italian Restaurant


In [49]:
Zug5_merged.loc[Zug5_merged['Cluster Labels'] == 1, Zug5_merged.columns[[2] + list(range(5, Zug5_merged.shape[1]))]]

,Gemeinden,Rent Yearly,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Walchwil,370,47.0996,8.51657,1,Train Station,Grocery Store,Restaurant,Gas Station,College Gym


In [50]:
Zug5_merged.loc[Zug5_merged['Cluster Labels'] == 2, Zug5_merged.columns[[2] + list(range(5, Zug5_merged.shape[1]))]]

,Gemeinden,Rent Yearly,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Risch,660,47.13474,8.46636,2,Lake,Bowling Alley,Hotel,Restaurant,Seafood Restaurant
